In [1]:
import os

from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from dotenv import load_dotenv
import pandas as pd

In [2]:
load_dotenv()

True

In [3]:
all_pages = []

DIRPATH = "./data/pdfs/"

for filename in os.listdir(DIRPATH):
    filepath = os.path.join(DIRPATH, filename)
    loader = PyPDFLoader(filepath)
    pages = loader.load_and_split()
    all_pages.extend(pages)

In [4]:
embeddings = OpenAIEmbeddings()
vectors = FAISS.from_documents(all_pages, embeddings)

In [5]:
def search_similar(query: str):
    similars = vectors.similarity_search(query, k=3)
    
    return [s.page_content for s in similars]

In [6]:
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k")

prompt = PromptTemplate(
    input_variables=["contents", "query"],
    template="""
        You are an expert academic researcher that enjoys helping other researchers, like myself.
        Based on the following content from research papers that I am familiar with and I know are related to my question: 
        
        {contents}

        {query}
        """
)

chain = LLMChain(llm=model, prompt=prompt)

In [7]:
def reply(query: str):
    similars = search_similar(query)
    response = chain.run(contents=similars, query=query)

    return response

In [12]:
reply("Who are the authors that addressed the alpha-neighbor p-center problem?")

'The authors that addressed the alpha-neighbor p-center problem are J. Sánchez-Oro, A.D. López-Sánchez, A.G. Hernández-Díaz, et al.'

In [9]:
reply("Based on the papers, who is the president of the United States?")

'Based on the content provided, there is no information about the president of the United States. The research papers mentioned are related to the topics of local search procedures, tabu search, and the α−pCP problem.'

In [10]:
reply("Who is the first lady of Mexico?")

'The information provided does not contain any relevant information about the first lady of Mexico.'

In [11]:
reply("What is the main feature of the Fast Vertex Substitution and how can I adapt it to the alpha-neighbor p-center problem?")

'The main feature of the Fast Vertex Substitution is its ability to efficiently evaluate and update the objective function value during the vertex substitution process. This is achieved by finding the best deletion (facility to be removed) during the evaluation of the objective function value, reducing the complexity of the procedure.\n\nTo adapt the Fast Vertex Substitution to the alpha-neighbor p-center problem, you can incorporate the concept of alpha-neighborhood into the procedure. The alpha-neighborhood of a solution X is defined as the set of solutions that can be obtained by replacing at most alpha fraction of the facilities in X with facilities outside of X.\n\nIn the Fast Vertex Substitution, instead of considering all possible replacements of facilities, you can limit the replacements to only those that result in solutions within the alpha-neighborhood of X. This can be done by modifying the Move procedure to consider only those replacements that satisfy the alpha-neighborho